In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# all columns
# name scheme: columns-num_cycles-sl-noise
from config import *
context = Context('2-23-redundent-features', fn_cycles='tbmData/cycles1', debug=0, mulr=0, load_data=False)

In [ ]:
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation

# idx = fmtr.beginning_index()
# df_conts, stat_x = fmtr.get_x()
# deps, stat_y = fmtr.get_y(normalize=True)
# train_cont = flatten_and_cat(df_conts, deps, sl=context.sl)

In [ ]:
std = cycles.std()

In [ ]:
std.shape, cycles.shape

In [ ]:
std[:5], cycles.columns[:5]

In [ ]:
cycles.drop(columns='运行时间', inplace=True)
cycles.drop(columns=cycles.columns[std == 0], inplace=True)

In [ ]:
cycles.shape

In [ ]:
from scipy.cluster import hierarchy as hc
corr = np.round(scipy.stats.spearmanr(cycles).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')

In [ ]:
def distance(c1, c2):
    columns = cycles.columns.tolist()
    print(corr[columns.index(c1), columns.index(c2)])
distance('时间戳', '刀盘运行时间')

In [ ]:
fig = plt.figure(figsize=(30,70))
dendrogram = hc.dendrogram(z, labels=cycles.columns, 
      orientation='left', leaf_font_size=20)
set_ax_font(fig.axes[0])
# ax.plot()
plt.savefig('spearmanr-dendrogram.pdf')

find the correlated variables with 时间戳 and exclude that from being dropped

Nope, this chance is gone

运行时间

In [ ]:
# drop sensors 2#, 6#, 8#, preserving 10#
to_drop = ['主驱动2#电机输出功率', '主驱动6#电机输出功率', '主驱动8#电机输出功率', '主驱动2#电机电流', '主驱动5#电机电流', '主驱动8#电机电流', '主驱动2#电机扭矩', '主驱动6#电机扭矩', '主驱动8#电机扭矩', '主驱动2#电机输出频率', '主驱动6#电机输出频率', '主驱动8#电机输出频率',
                     '减速机5#温度', '减速机4#温度', '减速机9#温度', '减速机3#温度', '减速机2#温度', '减速机7#温度', '减速机6#温度', '减速机8#温度', # drop anything other than 10# to be consistant with above
                     '主驱动3#电机输出频率', '主驱动3#电机输出功率', '主驱动3#电机扭矩', 
                     # drop 7#, 5#, preserving 9
                     '主驱动7#电机扭矩', '主驱动5#电机扭矩', '主驱动7#电机输出功率', '主驱动5#电机输出功率', '主驱动7#电机电流', '主驱动5#电机电流', '主驱动7#电机输出频率', '主驱动5#电机输出频率',
                     '推进速度.1', '推进给定速度百分比', '右推进油缸行程检测', '刀盘运行时间', #'时间戳'?
                     '二次风机频率设置', '推进压力', '刀盘给定转速显示值', '刀盘转速电位器设定值', '变频柜回水温度报警值', '拖车尾部CH4浓度', '污水箱压力检测']
[o for o in to_drop if o not in cycles.columns] # sanity check

In [ ]:
def evaluate_and_score():
    context = Context('removed_redundent2-full-30-7', gpu_start=8, dep_var=dep_var1, debug=0, mulr=7)
    databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)
    